In [1]:
from __future__ import print_function
import requests
import pandas as pd
import numpy as np

In [2]:
kewy_list="T:\colin\BRice\V2\gwas-gene-discovery\mock_keyword_list.txt"
disc_genes="T:\colin\BRice\V2\gwas-gene-discovery\MLM.DTF.GWAS.Results\summary_genes_discovered.txt"

In [67]:
def knetapi(disc_genes):
    with open(kewy_list, "r") as fl:
        pheno=[]
        for line in fl:
            pheno.append(line.rstrip())
            #all keyword from plain text file into pheno list.
        summary=pd.read_csv(disc_genes, sep="\t")
        genelist=list(summary[u'GENE'])
        genestr=(",").join(genelist)
        #all genes in column 0 of summary into one string
        
        #use str.join() to convert multiple elments in a list into one string.
        keyw1 = "%20OR%20".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
        link="http://knetminer.rothamsted.ac.uk/riceknet/genome?"
        parameters={"keyword":keyw1, "list":genestr}
        r=requests.get(link, params=parameters)
        
        #check if requests is successful
        if not r.ok:
                r.raise_for_status()
                sys.exit()

        #extract unicode string of geneTable decoded from json
        decoded=r.json()[u'geneTable'].split("\t")
        #remove space or newline at the end
        decoded=(decoded)[:-1]
        
        colnum=9
        #tabulate genetable into 9 columns.
        genetable=np.array(decoded).reshape(len(decoded)//colnum, colnum)
        genetable=pd.DataFrame(genetable[1:,:], columns=genetable[0,:])
        
        #delete useless columns from genetable.
        genetable=genetable.drop([u'ONDEX-ID', u'GENE NAME', u'TAXID', u'USER', u'QTL'], axis=1)
        
        #rename genetable columns.
        genetable.rename (
        columns={
            "ACCESSION":"Knetminer_genes",
            "SCORE":"KnetScore"
        }, inplace=True
        )
        
        knetgenes=list(genetable[u'Knetminer_genes'])
        knetscores=list(genetable[u'KnetScore'])
        
        #print(knetgenes)
        #print(knetscores)
        
        knetdict=dict(zip(knetgenes, knetscores))
        #print(knetdict)
        #print(knetdict[u'OS03G0122600'])
        #compare genetable and summary
        
        for i in summary[u'GENE']:
            i=i.upper()
            print(i)
            print(knetdict[u'{}'.format(i)])

In [68]:
knetapi(disc_genes)

OS03G0122600
125.60
OS03G0122600
125.60
OS03G0122600
125.60
OS03G0122600
125.60
OS03G0122600
125.60
OS06G0164900
5.94
OS03G0122600
125.60
OS06G0166100
0.00
OS06G0156700
10.64
OS06G0156600
10.64
